In [1]:
gt_dataset_id = 88850
dt_dataset_id = 89661
save_path = 'data/model-benchmark'

In [ ]:
import supervisely as sly
import json
import os
from model_benchmark import sly2coco

api = sly.Api()

gt_dataset_info = api.dataset.get_info_by_id(gt_dataset_id)
gt_dataset_name = gt_dataset_info.name
gt_project_id = gt_dataset_info.project_id
gt_project_info = api.project.get_info_by_id(gt_project_id)
gt_project_name = gt_project_info.name

dt_dataset_info = api.dataset.get_info_by_id(dt_dataset_id)
dt_dataset_name = dt_dataset_info.name
dt_project_id = dt_dataset_info.project_id
dt_project_info = api.project.get_info_by_id(dt_project_id)
dt_project_name = dt_project_info.name

assert gt_dataset_name == dt_dataset_name

save_project_path = os.path.join(save_path, dt_project_name)
gt_project_path = os.path.join(save_project_path, "gt_dataset")
dt_project_path = os.path.join(save_project_path, "dt_dataset")

# Download project
sly.download(api, gt_project_id, gt_project_path, dataset_ids=[gt_dataset_id], log_progress=True, save_images=False)
sly.download(api, dt_project_id, dt_dataset_id, dataset_ids=[dt_dataset_id], log_progress=True, save_images=False)

In [12]:
# from model_benchmark import sly2coco
# from importlib import reload
# reload(sly2coco)

<module 'model_benchmark.sly2coco' from '/root/model-benchmark/model_benchmark/sly2coco.py'>

In [28]:
# %%time
cocoGt = sly2coco.sly2coco(gt_project_path, gt_dataset_name, is_dt_dataset=False)
cocoDt = sly2coco.sly2coco(dt_project_path, dt_dataset_name, is_dt_dataset=True)

assert cocoDt['categories'] == cocoGt['categories']
assert all([img1['file_name'] == img1['file_name'] for img1, img2 in zip(cocoGt['images'], cocoDt['images'])])

In [29]:
cocoGt_path = os.path.join(save_project_path, "cocoGt.json")
cocoDt_path = os.path.join(save_project_path, "cocoDt.json")
with open(cocoGt_path, 'w') as f:
    json.dump(cocoGt, f)
with open(cocoDt_path, 'w') as f:
    json.dump(cocoDt['annotations'], f)

In [ ]:
# rtdetr-50 ep200: 0.177
# rtdetr-50 ep60: 0.213
# rtdetr-50 EMA ep60: 0.189
# rtdetr-50 MODEL ep60: 0.188